Setting working directory

In [3]:
import os

In [4]:
os.chdir("C:/Users/Ábel/Desktop/Prog_nyelvek/ProjectWork/Algorithms/")

Importing request for news requests

In [5]:
import requests

In [6]:
from newsapi import NewsApiClient

Datetime for giving start and end date parameters

In [7]:
from datetime import datetime, timedelta

For serlization (writing and reading)

In [8]:
import json

In [9]:
api_key = "58dc66dc611848a694e6fdd04c66917a"  # Replace with your actual API key
news_api = NewsApiClient(api_key=api_key)

Function to fetch and store the articles (Sadly, this method is deprecated now there is a limit in the dev. requests ( max of 100 results))

In [10]:
def fetch_and_store_articles_old(keyword, start_date, end_date, output_file):

    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")

    response = news_api.get_everything(
        q=keyword,
        from_param=start_date_str,
        to=end_date_str,
        sort_by='publishedAt' 
    )

    if response["status"] == "ok":
        if response["totalResults"] > 0:
            articles_list = []
            for article in response["articles"]:
                article_details = {
                    "title": article["title"],
                    "description": article["description"],
                    "url": article["url"],
                    "published_at": article["publishedAt"]
                }
                articles_list.append(article_details)

            with open(output_file, 'w') as json_file:
                json.dump(articles_list, json_file, indent=2)

            print(f"Articles successfully stored in {output_file}")
        else:
            print("No articles found.")
    else:
        print(f"Failed to fetch articles. Error: {response.get('message', 'Unknown error')}")


Function to fetch and store the articles (UpToDate)

In [11]:
def fetch_and_store_articles(keyword, start_date, end_date, output_file, interval_days=1):
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")

    articles_list = []

    current_date = start_date
    while current_date <= end_date:
        
        interval_end = min(current_date + timedelta(days=interval_days - 1), end_date)

        response = news_api.get_everything(
            q=keyword,
            from_param=current_date.strftime("%Y-%m-%d"),
            to=interval_end.strftime("%Y-%m-%d"),
            sort_by='publishedAt',
            page_size=100
        )

        if response["status"] == "ok":
            if response["totalResults"] > 0:
                for article in response["articles"]:
                    article_details = {
                        "title": article["title"],
                        "description": article["description"],
                        "url": article["url"],
                        "published_at": article["publishedAt"]
                    }
                    articles_list.append(article_details)
            else:
                print(f"No articles found from {current_date.strftime('%Y-%m-%d')} to {interval_end.strftime('%Y-%m-%d')}")

        else:
            print(f"Failed to fetch articles for {current_date.strftime('%Y-%m-%d')} to {interval_end.strftime('%Y-%m-%d')}. Error: {response.get('message', 'Unknown error')}")

        #Next interval
        current_date += timedelta(days=interval_days)

    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(articles_list, json_file, indent=2, ensure_ascii=False)

    print(f"Articles successfully stored in {output_file}. Total articles fetched: {len(articles_list)}")


Reading setup parametes from a text file

In [12]:
with open('parameters.txt', 'r') as file:
    lines = file.readlines()

In [ ]:
start_date = lines[0].strip()
start_date = datetime.strptime(start_date, '%Y-%m-%d').replace(hour=0, minute=0, second=0)
end_date = lines[1].strip()
end_date = datetime.strptime(end_date, '%Y-%m-%d').replace(hour=0, minute=0, second=0)
keyword = lines[2].strip()

Output file name

In [13]:
start_date_format = start_date.strftime("%Y-%m-%d")
end_date_format = end_date.strftime("%Y-%m-%d")

In [14]:
output_file = f"Data/BaseJson/all_articles_{keyword}_{start_date_format}_{end_date_format}.json"

Call the function to fetch and store the articles

In [15]:
fetch_and_store_articles(keyword, start_date, end_date, output_file)

Articles successfully stored in Data/BaseJson/all_articles_Berkshire Hathaway_2024-05-10_2024-05-10.json. Total articles fetched: 35


For better information

In [16]:
from bs4 import BeautifulSoup

Function to extract text from a given URL

In [17]:
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return text
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

Input Json for extraction

In [18]:
input_json_path = output_file

Output Json for extraction

In [30]:
output_json_path = f'Data/FullJson/fullout_all_articles_{keyword}_{start_date_format}_{end_date_format}.json'

Opening file

In [24]:
with open(input_json_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

Process each entry and store in a new JSON structure

In [25]:
output_data = []
for entry in json_data:
    title = entry["title"]
    description = entry["description"]
    published_at = entry["published_at"]

    text_content = extract_text_from_url(entry["url"])

    new_entry = {
        "title": title,
        "description": description,
        "published_at": published_at,
        "content": text_content
    }
    
    output_data.append(new_entry)

Error fetching content from https://www.kpbs.org/podcasts/kpbs-roundtable/student-journalists-role-in-telling-story-of-ucsd-encampment: 403 Client Error: Forbidden for url: https://www.kpbs.org/podcasts/kpbs-roundtable/student-journalists-role-in-telling-story-of-ucsd-encampment
Error fetching content from https://www.fool.com.au/2024/05/11/top-asx-shares-to-buy-for-your-superannuation-fund-in-may-2024/: 403 Client Error: Forbidden for url: https://www.fool.com.au/2024/05/11/top-asx-shares-to-buy-for-your-superannuation-fund-in-may-2024/
Error fetching content from https://www.marketscreener.com/quote/stock/JONES-LANG-LASALLE-INCORP-13180/news/Jones-Lang-LaSalle-Incorporated-Johns-Manville-signs-new-lease-to-keep-world-headquarters-in-Denve-46693274/: 403 Client Error: Forbidden. for url: https://www.marketscreener.com/quote/stock/JONES-LANG-LASALLE-INCORP-13180/news/Jones-Lang-LaSalle-Incorporated-Johns-Manville-signs-new-lease-to-keep-world-headquarters-in-Denve-46693274/
Error fetch

Write back 

In [31]:
with open(output_json_path, 'w', encoding='utf-8') as output_file:
    json.dump(output_data, output_file, indent=2, ensure_ascii=False)

Information:

In [32]:
print(f"Extraction complete. Results stored in {output_json_path}")

Extraction complete. Results stored in Data/FullJson/fullout_all_articles_Berkshire Hathaway_2024-05-10_2024-05-10.json


Importing csv for exporting out data

In [33]:
import csv

In [34]:
!pip3 install textblob

Importing textblob for data processing

In [35]:
from textblob import TextBlob

Load JSON data with Unicode decoding

In [36]:
with open(output_json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

Function to calculate sentiment score between 0 and 1

In [37]:
def calculate_sentiment_normalized(text):
    if text is None:
        text = ""  # Default to empty string if input is None
    analysis = TextBlob(text)
    sentiment_score = analysis.sentiment.polarity  # No need to scale to 10
    normalized_score = (sentiment_score + 1) / 2  # Normalize between 0 and 1
    return normalized_score

CSV Data

In [38]:
csv_data = []

Process each entry in the JSON data

In [39]:
for entry in data:
    title = entry['title']
    content = entry['content']
    description = entry['description']
    date = entry['published_at']

    if title == "[Removed]":
        continue

    if content is not None:
        title_sentiment = calculate_sentiment_normalized(title)
        content_sentiment = calculate_sentiment_normalized(content)

        if description is not None:
            description_sentiment = calculate_sentiment_normalized(description)
        else:
            description_sentiment = 0.5

        csv_data.append({
            'date': date,
            'title_sentiment': title_sentiment,
            'content_sentiment': content_sentiment,
            'description_sentiment': description_sentiment
        })
    else:

        csv_data.append({
            'date': date,
            'title_sentiment': 0.5,
            'content_sentiment': 0.5,  
            'description_sentiment': 0.5  
        })

csv file output

In [40]:
csv_file_path = f'Data/OutputSent/output_sentiments_normalized_{keyword}_{start_date_format}_{end_date_format}.csv'

Csv write back

In [41]:
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['date', 'title_sentiment', 'content_sentiment', 'description_sentiment']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()

    for entry in csv_data:
        writer.writerow({
            'date': entry['date'],
            'title_sentiment': entry['title_sentiment'],
            'content_sentiment': entry['content_sentiment'],
            'description_sentiment': entry['description_sentiment']
        })


Importing Pandas for creating dataFrame

In [42]:
import pandas as pd

Read the CSV file into a DataFrame

In [43]:
df = pd.read_csv(csv_file_path)

Convert the 'date' column to datetime

In [44]:
df['date'] = pd.to_datetime(df['date']).dt.date

Group by date and calculate the average for each sentiment column

In [45]:
avg_df = df.groupby('date').mean().reset_index()

Write the resulting DataFrame to a new CSV file

In [46]:
file_name=f'Data/AverageSent/average_sentiments_{keyword}_{start_date_format}_{end_date_format}.csv'

In [47]:
avg_df.to_csv(file_name, index=False)

Information:

In [48]:
print(f"Average sentiments saved to {file_name}")

Average sentiments saved to Data/AverageSent/average_sentiments_Berkshire Hathaway_2024-05-10_2024-05-10.csv
